# RMI Detail Transactions Project

## 1.  Load Transactional Data, load packages, and convert to Dataframes

I am using two reports from Netsuite to populate this data:  Lists/Employees --> Run report without filters all set to "All".

Search in Netsuite search bar usings "sea" for Adaptive Transaction Search - P&L - Jeremy

![title](img/picture.jpg)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from xml.sax import ContentHandler, parse

# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse('../Data/raw/ADAPTIVETransactionSearchPLJeremyResults849.xls', excelHandler)
df1 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])



In [3]:
df1.head()

,Date,Period,GL Account,Document Number,Program,Initiative,Project,Restriction,Grant,Amount,Vendor Name,Employee Name,Memo,Amount (Credit),Amount (Debit),Client,Transaction Number,Type,Vendor Rate,Vendor Contract Expiration Date
0,2020-07-01T00:00:00,Jul 2020,5010 Contributions Revenue : Foundation Contri...,JE5152,Development,DEV - Admin,Donation : Donation,2-Temporary Restricted,WRI - Islands FY20,50000.00,,,WRI - Islands FY20,50000.00,,,JOURNAL5152,Journal,,
1,2020-09-30T00:00:00,Sep 2020,5010 Contributions Revenue : Foundation Contri...,JE5424,Development,DEV - Admin,Donation : Donation,2-Temporary Restricted,WRI - Islands FY20,50000.00,,,WRI - Islands FY20,50000.00,,,JOURNAL5424,Journal,,
2,2020-10-31T00:00:00,Oct 2020,5010 Contributions Revenue : Foundation Contri...,JE5578,Development,DEV - Admin,Donation : Donation,2-Temporary Restricted,WRI - Islands FY20,150000.00,,,WRI - Islands FY20,150000.00,,,JOURNAL5578,Journal,,
3,2020-07-31T00:00:00,Jul 2020,7020 Salaries & Wages : Salaries & Wages,JE5137,Islands,IS - Islands,*Restricted : NDC Partnerships CAEP - Belize,1-Unrestricted,WRI - Islands FY20,242.53,,,JULY 2020 000626 Sidney Jules *Restricted : ND...,,242.53,,JOURNAL5137,Journal,,
4,2020-07-31T00:00:00,Jul 2020,7020 Salaries & Wages : Salaries & Wages,JE5137,Islands,IS - Islands,*Restricted : NDC Partnerships CAEP - Belize,1-Unrestricted,WRI - Islands FY20,2740.03,,,JULY 2020 000877 Shelley Backstrom *Restricted...,,2740.03,,JOURNAL5137,Journal,,


In [4]:
excelHandler = ExcelHandler()
parse('../Data/raw/Employees621.xls', excelHandler)
employee_list = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])


In [5]:
employee_list.head()

,Inactive,Internal ID,Name,Job Title,Program,Monthly Bonus Amount,Initiatives,Termination/Release Date,Hourly Rate,External ID
0,No,94538,Aaron Davitt,Principal Analyst,WattTime,,WAT - Admin,,,000978
1,No,45398,Aaron G Glazer,Associate,Carbon-Free Electricity,200.00,POW - Admin,,,000802
2,No,93272,Aaron Schwartz,Intern,Carbon-Free Electricity,,POW - Admin,2020-09-04T00:00:00,,000937
3,No,92131,Abhishek Jain,Intern,Carbon-Free Buildings,.00,BLD - Admin,2020-10-31T00:00:00,,000943
4,No,3793,Adam Klauber,Director,Industry,,INY - Admin,2019-05-17T00:00:00,,000524


## 2. Data Manipulation

In this section I start manipulting these datasets into somethign more digestable for the end user.  The main manipulation I am doing in this seciton is pulling information out of the "Memo" field that users may want to see seperately.

In [6]:
EmployeeNameandID = employee_list[['Name', 'External ID', 'Job Title']]

In [7]:
EmployeeNameandID['External ID'].replace('', np.nan, inplace=True)

C:\Users\Jeremy Wendt\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [8]:
len(EmployeeNameandID)

497

In [9]:
#EmployeeNameandID.to_excel('originallist.xlsx')

In [10]:
employee_id_list = employee_list['External ID'].tolist()

In [11]:
employee_id_list = list(filter(None, employee_id_list))

In [12]:
print(len(employee_id_list))

453


In [13]:
df1['Memo'] = df1['Memo'].astype('string')

In [14]:
df1.dtypes

Date                               object
Period                             object
GL Account                         object
Document Number                    object
Program                            object
Initiative                         object
Project                            object
Restriction                        object
Grant                              object
Amount                             object
Vendor Name                        object
Employee Name                      object
Memo                               string
Amount (Credit)                    object
Amount (Debit)                     object
Client                             object
Transaction Number                 object
Type                               object
Vendor Rate                        object
Vendor Contract Expiration Date    object
dtype: object

In [15]:
all_ids = '|'.join(employee_id_list)

In [16]:
all_ids

'000978|000802|000937|000943|000524|000631|001029|000952|000663|000981|000847|000945|000586|000620|000583|000622|000549|000550|000891|000977|000639|000563|000558|000447|000832|000913|000878|000935|000106|001042|000453|000902|000539|000409|000595|000614|000574|000831|001012|000658|000889|000973|000554|000926|000499|000502|000951|000907|000863|000931|000885|000850|000919|000846|000638|001030|000229|000967|000972|000629|000476|000914|000572|000918|000293|000893|001041|000936|000816|000879|000856|000635|000920|001028|000456|000131|000589|000827|000646|000566|000274|000569|000641|000826|000649|000851|000818|000912|000601|000437|000867|000580|000581|000403|000623|001017|000896|000643|000634|000472|000520|001022|000655|000815|000911|000984|000388|000895|000323|000624|000954|000849|000157|000823|000458|000890|000382|000428|000531|000479|001004|000396|000664|000400|000657|001033|000480|000616|000408|000840|000988|000291|000559|000908|000821|000363|000933|000604|000401|000613|000905|000215|00044

In [17]:
def return_first(x):
    try:
        y = x[0]
    except:
        y = ""
    return y

In [18]:
df1['External ID'] = df1['Memo'].str.findall(all_ids).apply(return_first)

In [19]:
employee_name_list = employee_list['Name'].tolist()

In [20]:
employee_name_list = list(filter(None, employee_name_list))

In [21]:
all_names = '|'.join(employee_name_list)

In [22]:
all_names

'Aaron Davitt|Aaron G Glazer|Aaron Schwartz|Abhishek Jain|Adam Klauber|Adam McKittrick|Adefunke Sonaike|Adriana Cid|Agnes Li|Ailin Asadinejad|Aimee Schwartze|Aishwarya Jaishankar|Akshay Krishnan|Alexander Engel|Alexander J Prince|Alexander Klonick|Alexandra Chin|Alexandra Rotatori|Alexandra Tsubota|Alexis Tubb|Ali Kargar|Alicia Noriega|Alisa Petersen|Allison Crow|Aman Chitkara|Amanda Farthing|Amar Shah|Ambrosia Gensterblum|Amit Halevi|Amory Lovins|Amos Zerah|Amy Baum|Amy Dalton|Amy Egerter|Amy Westervelt|Ana  S Mifsud|Andrew Keeton|Angela Whitney|Angelique Fathy|Anish Tilak|Ankit Kalanki|Anna  Ledden|Anna Palmer|Annabelle Sullivan|Annalise Grueter|Annis Benn|Anthony Teixeira|Aparna Ramesh|Aradhana Gahlaut|Ashley  Wiles|Ashley Gremel|Ashley Tate|Audrey Allen|Audrey Rempher|Avery McEvoy|Avril Levasseur|Ben  Shapiro|Ben Holland|Ben Miller|Ben Serrurier|Benjamin Miller|Bianca Wachtel|Billy Mwanda|Bingqi Liu|Blair Swift|Bonny Joseph|Brad Mushovic|Brady Seals|Brant W Richards|Brent Butler|Br

In [23]:
all_names = all_names + "|Agnis Li|Anna Ledden|Greg Hales|Rachel Hohensee|Li Ma"

In [24]:
df1['Match ID Names'] = df1['Memo'].str.findall(all_names).apply(return_first)

In [25]:
df1 = pd.merge(df1, EmployeeNameandID, on = 'External ID', how='left')

In [26]:
df1['Match ID Names'].replace('', np.nan, inplace=True)

In [27]:
df1['Name'].replace('', np.nan, inplace=True)

In [28]:
df1['Name'].fillna(df1['Match ID Names'], inplace=True)

In [29]:
df1['Name'].replace('Agnis Li', 'Agnes Li', inplace=True)
df1['Name'].replace('Anna Ledden', 'Anna  Ledden', inplace=True)
df1['Name'].replace('Rachel Hohensee', 'Rachel T Hohensee', inplace=True)

In [30]:
len(df1)

84467

## Extract Time Allocation From Memo

In [31]:
allocations = ['BONUS ALLOCATION', 'ADMIN ALLOCATION', 'PTO/HOLIDAY ALLOCATION']

In [32]:
all_allocations = '|'.join(allocations)

In [33]:
df1['ALLOCATION'] = df1['Memo'].str.findall(all_allocations).apply(return_first)

## Extract Project Fields for Heather's report

In [34]:
df1['Project Allocations'] = df1['Project'].str.findall(r'\[(.*?)\]').apply(return_first)

## Add Rollup Accounts

In [35]:
lookups = pd.read_excel('../Data/raw/lookups.xlsx')

In [36]:
lookups.head()

,GL Account,Rollup Account
0,5010 Contributions Revenue : Foundation Contri...,5000 Contributions Revenue
1,5015 Contributions Revenue : Campaign Foundati...,5000 Contributions Revenue
2,5020 Contributions Revenue : Individual Contri...,5000 Contributions Revenue
3,5025 Contributions Revenue : Campaign Corporat...,5000 Contributions Revenue
4,5030 Contributions Revenue : Endowment Contrib...,5000 Contributions Revenue


In [37]:
df1 = pd.merge(df1, lookups, on = 'GL Account', how='left')

In [38]:
df1['Account Digit'] = df1['GL Account'].astype('string').str[0]

In [39]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'External ID', 'Match ID Names',
       'Name', 'Job Title', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit'],
      dtype='object')

In [40]:
df1['Amount'] = df1['Amount'].astype(float)
df1['Amount (Credit)'].replace('', 0, inplace=True)
df1['Amount (Debit)'].replace('', 0, inplace=True)
df1['Amount (Credit)'] = df1['Amount (Credit)'].astype(float)
df1['Amount (Debit)'] = df1['Amount (Debit)'].astype(float)

In [41]:
df1.dtypes

Date                                object
Period                              object
GL Account                          object
Document Number                     object
Program                             object
Initiative                          object
Project                             object
Restriction                         object
Grant                               object
Amount                             float64
Vendor Name                         object
Employee Name                       object
Memo                                string
Amount (Credit)                    float64
Amount (Debit)                     float64
Client                              object
Transaction Number                  object
Type                                object
Vendor Rate                         object
Vendor Contract Expiration Date     object
External ID                         object
Match ID Names                      object
Name                                object
Job Title  

## Add Reclass Field and Closeout field

In [42]:
account_exceptions = ['To reclass', 'Reclass']

In [43]:
Accounting_Moves = '|'.join(account_exceptions)

In [44]:
closeout = 'close out'

In [45]:
df1['Accounting Adjustments'] = df1['Memo'].str.findall(Accounting_Moves).apply(return_first)

In [46]:
df1['Accounting Close Out'] = df1['Memo'].str.findall(closeout).apply(return_first)

In [47]:
df1['Vendor Contract Expiration Date'] = pd.to_datetime(df1['Vendor Contract Expiration Date'])

In [48]:
df1['Vendor Contract Expiration Date'] = df1['Vendor Contract Expiration Date'].dt.date

## ADDING TITLES INSTEAD OF NAMES

In [49]:
EmployeeNameandID['Job Title'].value_counts()

Senior Associate                  73
Manager                           63
Associate                         63
Intern                            47
Principal                         38
                                  ..
Help Desk Lead                     1
Data Reporting Analyst             1
Development Operations Manager     1
Email Marketing Lead               1
Controller                         1
Name: Job Title, Length: 111, dtype: int64

In [50]:
EmployeeNameandID.head()

,Name,External ID,Job Title
0,Aaron Davitt,000978,Principal Analyst
1,Aaron G Glazer,000802,Associate
2,Aaron Schwartz,000937,Intern
3,Abhishek Jain,000943,Intern
4,Adam Klauber,000524,Director


In [51]:
uniquetitles = EmployeeNameandID['Job Title'].unique()

In [52]:
Titleslist = uniquetitles.tolist()

In [53]:
def createEmployeeTitlesdf(title, df):
    newdf = df[df['Job Title']==title]
    newdf.reset_index(drop=True, inplace=True)
    newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)    
    return newdf.to_dict()

In [54]:
DictList = {}

for i in range(len(Titleslist)):
    newdict = createEmployeeTitlesdf(Titleslist[i], EmployeeNameandID)
    DictList[i] = newdict

<ipython-input-53-933c5a1db0ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)


In [55]:
dflist = []

for i in range(len(DictList)):
    df = pd.DataFrame(DictList[i])
    dflist.append(df)

In [56]:
finalemployeedf = pd.concat(dflist).reset_index(drop=True)

In [57]:
employeelisttomerge = finalemployeedf[['Name', 'PositionID']]

In [58]:
#employeelisttomerge.to_excel('listwithposition.xlsx')

In [59]:
len(employeelisttomerge)

497

In [60]:
df1 = pd.merge(df1, employeelisttomerge, on = 'Name', how='left')

In [61]:
df1['Initiative'].replace('', np.nan, inplace=True)
df1['Project'].replace('', np.nan, inplace=True)
df1['Grant'].replace('', np.nan, inplace=True)
df1['GL Account'].replace('', np.nan, inplace=True)
df1['Memo'].replace('', np.nan, inplace=True)
df1['Transaction Number'].replace('', np.nan, inplace=True)
df1['Vendor Name'].replace('', np.nan, inplace=True)
df1['Vendor Rate'].replace('', np.nan, inplace=True)
df1['Vendor Contract Expiration Date'].replace('', np.nan, inplace=True)
df1['Program'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('e', np.nan, inplace=True)
df1['Accounting Adjustments'].replace('To reclass', 'Reclass', inplace=True)

In [62]:
df1['Initiative'].fillna("No Initiative", inplace=True)
df1['Project'].fillna("No Project", inplace=True)
df1['Grant'].fillna("No Grant", inplace=True)
df1['GL Account'].fillna("No GL Account", inplace=True)
df1['Memo'].fillna("No Memo", inplace=True)
df1['Transaction Number'].fillna("No Transaction Number", inplace=True)
df1['Vendor Name'].fillna("No Vendor Name", inplace=True)
df1['Vendor Rate'].fillna("No Vendor Rate", inplace=True)
df1['Vendor Contract Expiration Date'].fillna("No Contract Expiration Date", inplace=True)
df1['Program'].fillna("No Program", inplace=True)
df1['Project Allocations'].fillna("Other", inplace=True)


## Add Further Heather Calcs

In [63]:
df1['Inst or Reclass'] = df1['Account Digit'] + df1['Accounting Adjustments']

In [64]:
df1['Inst or Reclass'].value_counts()

7           65736
8           11527
5            4446
6            1852
7Reclass      694
8Reclass      165
5Reclass       46
Name: Inst or Reclass, dtype: Int64

In [65]:
df1['Inst or Reclass'] = df1['Inst or Reclass'].map({'7': "Expense", '8': "Institutional Overhead", '5': "Revenue", '6': "Revenue", '7Reclass': "Expense Reclass", '8Reclass': "Inst Overhead Reclass", '5Reclass': "Revenue Reclass", '6Reclass': "Revenue Reclass"})

## Unrestricted Columns Added

In [66]:
def return_grant(x):
    if x != "No Grant":
        return "Yes"
    else: 
        return "No"
    
    

In [67]:
df1['grant?'] = df1['Grant'].apply(return_grant)

In [68]:
df1["Restricted Revenue"] = df1['grant?'] + df1['Account Digit']

In [69]:
df1["Restricted Revenue"] = df1["Restricted Revenue"].map({'No7': "No", 'Yes7': "Restricted Revenue", 'No8': "No", 'No5': "No", 'Yes8': "Restricted Revenue", 'Yes6': "No", 'No6': "No", 'Yes5': "No"})

In [70]:
df1['Restricted Revenue'] = np.where(df1['Restricted Revenue'] == "Restricted Revenue", df1['Amount'], 0)

In [71]:
df1["Rev or Exp"] = df1["Account Digit"].map({'6': 'Revenue', '5': 'Revenue', '7': 'Expense', '8': 'Expense'})

In [72]:
df1["Rev or Exp"].value_counts()

Expense    78122
Revenue     6344
Name: Rev or Exp, dtype: int64

In [73]:
df1['Multilateral'] = np.where(df1['Rev or Exp'] == "Revenue", df1['Amount'], 0)

In [74]:
df1['General Expense'] = np.where(df1['Rev or Exp'] == "Expense", df1['Amount'], 0)

In [75]:
df1['Unrestricted'] = - df1['Restricted Revenue'] - df1['Multilateral'] + df1['General Expense']

In [76]:
df1 = df1.drop('Match ID Names',1)

## Output manipulated DF to Excel

In [78]:
df1.to_excel('Output.xlsx')

# 3.  Data Segmentation by Program

Here I'm taking the manipulated data and ultimately creating a dataframe to filter out each program's restricted revenue.

In [79]:
Programs = list(df1['Program'].unique())

In [80]:
print(Programs)

['Development', 'Islands', 'India', 'Operations', 'China', 'Breakthrough Technology', 'Carbon-Free Mobility', 'Climate Aligned Industries', 'Global Climate Finance', 'Carbon-Free Electricity', 'Strategic Engagement and Analysis Group', 'Carbon-Free Buildings', 'WattTime', 'Urban Transformation', 'Climate Intelligence', 'Industry', 'Africa', 'SE Asia', 'US', 'Building Electrification', 'Communications', 'Energy Transition Academy', 'No Program']


In [81]:
def create_program(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[firstdf['Grant'] != 'No Grant'] 
    thirddf = seconddf[(seconddf['Account Digit']=='7')|(seconddf['Account Digit']=='8')]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf

In [82]:
Development = create_program('Development')
Islands = create_program('Islands')
India = create_program('India')
Operations = create_program('Operations')
China = create_program('China')
Breakthrough_Technology = create_program('Breakthrough Technology')
CarbonFree_Mobility = create_program('Carbon-Free Mobility')
Climate_Aligned_Industries = create_program('Climate Aligned Industries')
Global_Climate_Finance = create_program('Global Climate Finance')
Strategic_Engagement_and_Analysis_Group = create_program('Strategic Engagement and Analysis Group')
CarbonFree_Buildings = create_program('Carbon-Free Buildings')
WattTime = create_program('WattTime')
CarbonFree_Electricty = create_program('Carbon-Free Electricity')
Urban_Transformation = create_program('Urban Transformation')
Climate_Intelligence = create_program('Climate Intelligence')
Industry = create_program('Industry')
SE_Asia = create_program('SE Asia')
Africa = create_program('Africa')
US = create_program('US')
Building_Electrification = create_program('Building Electrification')



Development DF is EMPTY


In [83]:
Developing_Economies_Rollup = pd.concat([Islands, Africa, SE_Asia], ignore_index = True, sort=False)
CarbonFreeBuildings_Rollup = pd.concat([CarbonFree_Buildings, Building_Electrification], ignore_index = True, sort=False)

## 4. Create Reports Path and Manipulate File Names

I will need to come up with naming conventions that work over time for this project, so in this step I do just that.   Also I am loading the Openpyxl packages that i'm going to need for the rest of this program.

In [84]:
import os

path = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\Detail_Trans_Reports'

In [85]:
from datetime import datetime
today = datetime.today()
datem = datetime(today.year, today.month-1, 1)
timestamp = datem.strftime("%b_%Y")

In [86]:
print(timestamp+".xlsx")

Jan_2021.xlsx


In [87]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

## 5.  Create Pivot Table, Add Subtotals, Export to Excel, and Combine All These Functions into One

In [88]:
def create_multi_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Transaction Number', 'Vendor Name', 'Vendor Rate', 'Vendor Contract Expiration Date'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [89]:
def add_subtotal_multi(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [90]:
def to_excel(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,9), engine = 'openpyxl', sheet_name = 'Restricted_Rev_Detail', float_format = "%.2f")

In [91]:
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


def edit_workbook(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Transaction Number"
    sheet['G1']= "Vendor Name"
    sheet['H1']= "Vendor Rate"
    sheet['I1']= "Vendor Contract Expiration Date" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet['H1'].font = Font(bold=True)
    sheet['I1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(10, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 10):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Subtotal":
            for j in range(10,23):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
#    sheet.protection.sheet = True
    book.save(file)

In [92]:
def format_multi_pivot(Group, str):
    step1 = create_multi_pivot(Group)
    step2 = add_subtotal_multi(step1)
    to_excel(step2, str)
    edit_workbook(path+str)

## 6.  Create Restricted Revenue Files

In [93]:
%%time 
format_multi_pivot(Islands, '\\Islands_'+timestamp+".xlsx")
format_multi_pivot(India, '\\India_'+timestamp+".xlsx")
format_multi_pivot(Operations, '\\Operations_'+timestamp+".xlsx")
format_multi_pivot(China, '\\China_'+timestamp+".xlsx")
format_multi_pivot(Breakthrough_Technology, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Mobility, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_multi_pivot(Climate_Aligned_Industries, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_multi_pivot(Global_Climate_Finance, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_multi_pivot(Strategic_Engagement_and_Analysis_Group, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Buildings, '\\CarbonFree_Buildings_'+timestamp+".xlsx")


Wall time: 37.9 s


In [94]:
format_multi_pivot(WattTime, '\\WattTime_'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Electricty, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_multi_pivot(Urban_Transformation, '\\Urban_Transformation_'+timestamp+".xlsx")
format_multi_pivot(Climate_Intelligence, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_multi_pivot(Industry, '\\Industry_'+timestamp+".xlsx")
format_multi_pivot(SE_Asia, '\\SE_Asia_'+timestamp+".xlsx")
format_multi_pivot(Africa, '\\Africa_'+timestamp+".xlsx")
format_multi_pivot(US, '\\US_'+timestamp+".xlsx")
format_multi_pivot(Building_Electrification, '\\Building_Electrification_'+timestamp+".xlsx")          

## 7.  Add ML and Consulting Fee Tab

In [95]:
def create_df_FeeforService(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='5')|(firstdf['Account Digit']=='6')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [96]:
DevelopmentFFS = create_df_FeeforService("Development")
IslandsFFS = create_df_FeeforService("Islands")
IndiaFFS = create_df_FeeforService("India")
OperationsFFS = create_df_FeeforService("Operations")
ChinaFFS = create_df_FeeforService("China")
Breakthrough_Technology_FFS = create_df_FeeforService("Breakthrough Technology")
CarbonFree_MobilityFFS = create_df_FeeforService("Carbon-Free Mobility")
ClimateAligned_IndustriesFFS = create_df_FeeforService("Climate Aligned Industries")
CommunicationsFFS = create_df_FeeforService("Communications")
GlobalClimate_FinanceFFS = create_df_FeeforService("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupFFS = create_df_FeeforService("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsFFS = create_df_FeeforService("Carbon-Free Buildings")
Watttime_FFS = create_df_FeeforService("WattTime")
Carbon_Free_ElectricityFFS = create_df_FeeforService("Carbon-Free Electricity")
UrbanTrans_FFS = create_df_FeeforService("Urban Transformation")
ClimateIntelFFS = create_df_FeeforService("Climate Intelligence")
IndustryFFS = create_df_FeeforService("Industry")
SEAsiaFFS = create_df_FeeforService("SE Asia")
AfricaFFS = create_df_FeeforService("Africa")
USFFS = create_df_FeeforService("US")
BuildingElectrificationFFS = create_df_FeeforService("Building Electrification")

Communications DF is EMPTY


In [97]:
def create_FFS_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Client', 'Transaction Number'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [98]:
def add_subtotal_FFS(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '',  'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '',  '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [99]:
import openpyxl

def to_excel_FFS(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="ML & FFS Revenue")
    writer.save()
    writer.close()    

In [100]:
def edit_workbook_FFS(file):
    book = load_workbook(file)
    sheet = book["ML & FFS Revenue"] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Client"
    sheet['G1']= "Transaction Number" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(8, 21):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 8):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=7).value == "Subtotal":
            for j in range(8,21):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "H2"
    book.save(file)

In [101]:
def format_FFS_pivot(Group, str):
    step1 = create_FFS_pivot(Group)
    step2 = add_subtotal_FFS(step1)
    to_excel_FFS(step2, path+str)
    edit_workbook_FFS(path+str)

## 8.  Add ML and FFS Pivots to Second Tab

In [102]:
format_FFS_pivot(IslandsFFS, '\\Islands_'+timestamp+".xlsx")
format_FFS_pivot(IndiaFFS, '\\India_'+timestamp+".xlsx")
format_FFS_pivot(OperationsFFS, '\\Operations_'+timestamp+".xlsx")
format_FFS_pivot(ChinaFFS, '\\China_'+timestamp+".xlsx")
format_FFS_pivot(Breakthrough_Technology_FFS, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_FFS_pivot(CarbonFree_MobilityFFS, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_FFS_pivot(ClimateAligned_IndustriesFFS, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_FFS_pivot(GlobalClimate_FinanceFFS, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_FFS_pivot(Strategic_Engagement_and_Analysis_GroupFFS, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_FFS_pivot(Carbon_Free_BuildingsFFS, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_FFS_pivot(Watttime_FFS, '\\WattTime_'+timestamp+".xlsx")
format_FFS_pivot(Carbon_Free_ElectricityFFS, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_FFS_pivot(UrbanTrans_FFS, '\\Urban_Transformation_'+timestamp+".xlsx")
format_FFS_pivot(ClimateIntelFFS, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_FFS_pivot(IndustryFFS, '\\Industry_'+timestamp+".xlsx")
format_FFS_pivot(SEAsiaFFS, '\\SE_Asia_'+timestamp+".xlsx")
format_FFS_pivot(AfricaFFS, '\\Africa_'+timestamp+".xlsx")
format_FFS_pivot(USFFS, '\\US_'+timestamp+".xlsx")
format_FFS_pivot(BuildingElectrificationFFS, '\\Building_Electrification_'+timestamp+".xlsx")

# Create Reports for Heather and Jon Creyts

In [117]:
CFOpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\CFO_Reports'

## Restricted Revenue Spend Type by Grant

In [118]:
def create_CFO_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Grant','Inst or Reclass'], values = 'Restricted Revenue', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [119]:
def add_subtotal_CFO(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Grant')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [120]:
def to_excel_CFO(pivot, str):
    pivot.to_excel(CFOpath+str, freeze_panes = (1,2), engine = 'openpyxl', sheet_name = 'Expense Type by Grant', float_format = "%.2f")

In [121]:
def edit_workbook_CFO(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Grant"
    sheet['B1']= "Expense Type"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(3, 16):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(3,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
#    sheet.protection.sheet = True
    book.save(file)

In [122]:
def format_multi_pivot_CFO(Group, str):
    step1 = create_CFO_pivot(Group)
    step2 = add_subtotal_CFO(step1)
    to_excel_CFO(step2, str)
    edit_workbook_CFO(CFOpath+str)

In [123]:
dfRestrictedONLY = df1[df1['Rev or Exp']=="Expense"]

In [124]:
format_multi_pivot_CFO(dfRestrictedONLY, '\\CFO_Report_'+timestamp+".xlsx")

## Unrestricted Use by Project Allocation and Program

In [125]:
def create_CFO_pivot2(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Project Allocations','Program'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [126]:
def add_subtotal_CFO2(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Project Allocations')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [127]:
def to_excel_CFO2(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted by Project All")
    writer.save()
    writer.close()    

In [128]:
def edit_workbook_CFO2(file):
    book = load_workbook(file)
    sheet = book["Unrestricted by Project All"] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
 
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(3, 16):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(3,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "B2"
    book.save(file)

In [129]:
def format_multi_pivot_CFO2(Group, str):
    step1 = create_CFO_pivot2(Group)
    step2 = add_subtotal_CFO2(step1)
    to_excel_CFO2(step2, CFOpath+str)
    edit_workbook_CFO2(CFOpath+str)

In [130]:
format_multi_pivot_CFO2(df1, '\\CFO_Report_'+timestamp+".xlsx")